In [ ]:
from bs4 import BeautifulSoup
import requests
import base64
import pymongo
from collections import Counter
import re
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
connection_string="mongodb://localhost:27017"
yapraksila_ozturk=pymongo.MongoClient(connection_string)
database=yapraksila_ozturk["scrapeDB"]
collection=database["news"]

In [ ]:
data=[]
data_of={}

In [ ]:
collection=database["word_frequency"]
word_data=[]


In [ ]:
url="https://turkishnetworktimes.com/kategori/gundem/"
response=requests.get(url)
response=response.content
response
soup=BeautifulSoup(response, 'html.parser')

In [ ]:
def mostFrequentWords(all_text):
    words = re.findall('\w+', all_text)
    common=Counter(words).most_common(10)
    word_data.append(common)

In [ ]:
def extractData(get_url):
    url=get_url
    response=requests.get(url)
    response=response.content
    soup=BeautifulSoup(response, 'html.parser')
    inside=soup.find("div", {"class":"yazi_icerik"})
    text_inside=inside.find_all("p")
    for i in text_inside:
        data_of["text"]=i.text_inside
        #print(i.text)
        all_text=" "+i.text
    date=soup.find_all("span", {"class":"tarih"})
    for i in date:
        publish=i.find("time").get("datetime")
        update=i.find_next("time").get("datetime")
    data_of["publish_date"]=publish
    data_of["update_date"]=update
    data.append(data_of)
    mostFrequentWords(all_text)
    #print(publish)

In [ ]:
#get all news from each page and extract data
def getEachNews(soup):
    page=soup.find("div", {"class":"kategori_yazilist"})
    page=page.find_all("article", {"class":"col-12"})
    for i in page:
        get_url=i.find('a',{"class":"post-link"})['href']
        get_img=i.find('img').get('src')
        strOne = get_img.partition(",")[2]
        encoded_image = base64.b64decode(strOne)
        header=i.find('h2', {"class":"haber-baslik"}).text
        #print(header)
        data_of["url"]=get_url
        data_of["header"]=header
        data_of["image_url_list"]=encoded_image
        extractData(get_url)
        data.append(data_of)

In [ ]:
#find the first 50 pages
url="https://turkishnetworktimes.com/kategori/gundem/"
i=0
data=[]
data_of={}
while(i<49):
    if(i==0):
        response=requests.get(url)
    else:
        response=requests.get(next_url)
    response=response.content
    soup=BeautifulSoup(response, 'html.parser', from_encoding="utf-8")
    getEachNews(soup)
    
    next_page=soup.find("a", {"class":"next page-numbers"})
    if next_page:
        next_url=next_page.get("href")
        #print(next_url)
        i+=1

In [ ]:
#insert data into database
for i in data:
   x=collection.insert_one(i)